In [ ]:
from  lib.project_functions import *

In [ ]:
def load_csv_file(file_list):
    """
    Load CSV files into pandas DataFrames and assign them as global variables.

    Args:
        file_list (list): A list of file names to load as CSV files.

    Returns:
        None

    This function iterates through the provided list of file names and loads each CSV file
    into a pandas DataFrame. The DataFrames are assigned as global variables with names
    following the pattern 'csv_<file_name_without_extension>'.

    Note:
    - Ensure you have the required library (pandas) imported before using this function.
    - The function will only load files with a '.csv' extension from the specified list.
    - Global variables with names based on the loaded file names will be created, allowing
      access to the DataFrames globally.
    """
    for file in file_list:
        if ".csv" in file:
            # Extract the base file name without extension
            name_file = file.split(".")
            
            # Read the CSV file and assign it as a global variable
            globals()[f"csv_{name_file[0]}"] = pd.read_csv(f"data/{file}") 

In [ ]:
file_list = sorted(os.listdir("data"))
file_list

In [ ]:
load_csv_file(file_list)

In [ ]:
csv_Train.sample(5)

In [ ]:
csv_Train.nunique()

In [ ]:
classid_count = csv_Train['ClassId'].value_counts()
classid_count

In [ ]:
plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
plt.bar(classid_count.index, classid_count.values)
plt.xlabel('ClassId')
plt.ylabel('Count')
plt.title('Value Counts of ClassId')
plt.xticks(rotation=90)  # Rotate x-axis labels for better readability if necessary
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
csv_Train.describe()

In [ ]:
file_list

In [ ]:
data = loading_random_data()
data.shape

In [ ]:
for image in data:
    print(image.min(), image.max())
    image_show(image)
    show_histogram(image)

In [ ]:
for image in data:
    image = histogram_equalization(image)
    image_show( histogram_equalization(image))
    show_histogram(image)

In [ ]:
for image in data:
    image =  contour_alignment(image)
    image_show(image)
    show_histogram(image)

In [ ]:
for image in data:
    image = cv.medianBlur(image, 5)
    image_show(image)
    # show_histogram(image)

In [ ]:
for image in data:
    image_show(contour_in_binarized_image(image))

In [ ]:
for image in data:
    image_show(otsu_threshold_contours(image))
    image_show(adaptive_threshold_contours(image))

In [ ]:
for image in data:
    image = cv.medianBlur(image, ksize = 5) 
    image = histogram_equalization(image)
    image = contour_alignment(image, tileGridSize=(8, 8))
    image = contour_in_binarized_image(image)
    image = otsu_threshold_contours(image)
    # image = adaptive_threshold_contours(image)
    
    image_show(image)

In [ ]:
def median_blur(image, ksize = 5):
    return cv.medianBlur(image, ksize)

In [ ]:
# import itertools

# functions = {
#     "medianBlur": median_blur,
#     "histogram_equalization": histogram_equalization,
#     "contour_alignment": contour_alignment,
#     "contour_in_binarized_image": contour_in_binarized_image,
#     "otsu_threshold_contours": otsu_threshold_contours
# }


# function_combinations = []
# for i in range(len(functions) + 1 ):
#     function_combinations.extend(list(itertools.combinations(functions.keys(), i)))

# # Loop through each feature combination and each image
# for combination in function_combinations:
#     for image in data:
#         # Call each function by its name in the current combination
#         processed_image = image
#         for func_name in combination:
#             processed_image = functions[func_name](processed_image)
        
#         # Print information about the current combination of functions
#         print(f"Combination: {', '.join(combination)}")
        
       
#         image_show(processed_image)


 # favorite
 

In [ ]:
def image_preprocessing(image):
    image = cv.detailEnhance(image, sigma_s=50, sigma_r=0.3)
    image = cv.medianBlur(image, 3)
    image = cv.addWeighted(image, 1.5, image, -0.5, 0)
    # image = otsu_threshold_contours(image)
    
    return image

for image in data:
    image_show(image)
    image = image_preprocessing(image)
    image_show(image)